# Importing necessary libraries

In [ ]:
# # for data manipulation
# import pandas as pd

# # For visualization
# import matplotlib.pyplot as pyplot
# import seaborn as sns

# # for date/time manipulationimport datetime

# #fot nlp
# import nltk
# from nltk.sentiment import SentimentIntensityAnalyzer
# from nltk.tokenize import word_tokenize
# from nltk.corpus import stopwords
# from nltk.stem import PorterStemmer

# # for numerical analysis
# import numpy

In [14]:
# Import necessary modules
import pandas as pd
from scripts.data_loading import load_stock_data
from scripts.data_validation import validate_data_structure
from scripts.technical_analysis import calculate_technical_indicators
from scripts.financial_metrics import fetch_financial_metrics
from scripts.data_visualization import plot_stock_data, plot_rsi, plot_macd


ModuleNotFoundError: No module named 'scripts.data_loading'

In [12]:
import sys, os
#import os

# Add the project root directory to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

In [13]:
# Import necessary modules
from scripts.dataloader import load_csv, load_yfinance_data


# Define file paths
analyst_ratings_path = r"C:\Users\HP\Desktop\week - 1\Data\raw_analyst_ratings\raw_analyst_ratings.csv"
yfinance_folder_path = r"C:\Users\HP\Desktop\week - 1\Data\yfinance_data"

# Load data
raw_analyst_ratings = load_csv(analyst_ratings_path)
yfinance_data = load_yfinance_data(yfinance_folder_path)

# Inspect Raw Analyst Ratings
print("\nRaw Analyst Ratings Info:")
print(raw_analyst_ratings.info())
print(raw_analyst_ratings.head())

# Inspect YFinance Data
print("\nYFinance Data Keys (Stocks):", list(yfinance_data.keys()))
if 'AAPL_historical_data' in yfinance_data:
    print("\nSample AAPL Data:")
    print(yfinance_data['AAPL_historical_data'].head())


ModuleNotFoundError: No module named 'scripts.dataloader'

Loading Data

In [2]:
# import os
# import pandas as pd

# def load_data():
#     """
#     Load all data files from the 'yfinance_data' and 'raw_analyst_ratings' folders.
    
#     Returns:
#         stock_data (dict): A dictionary containing stock data DataFrames.
#         raw_analyst_ratings (pd.DataFrame): DataFrame containing raw analyst ratings.
#     """
#     # Define the paths
#     yfinance_folder = r'C:\Users\HP\Desktop\week - 1\Data\yfinance_data'
#     analyst_ratings_file = r'C:\Users\HP\Desktop\week - 1\Data\raw_analyst_ratings\raw_analyst_ratings.csv'
    
#     # Initialize a dictionary to store stock DataFrames
#     stock_data = {}
#     raw_analyst_ratings = None
    
#     # Load yfinance data files
#     if os.path.exists(yfinance_folder):
#         yfinance_files = [f for f in os.listdir(yfinance_folder) if f.endswith('.csv')]
#         for file in yfinance_files:
#             file_path = os.path.join(yfinance_folder, file)
#             stock_name = os.path.splitext(file)[0]  # Extract stock name without extension
#             print(f"Loading file: {file} as stock: {stock_name}")
#             stock_data[stock_name] = pd.read_csv(file_path)
    
#     # Load raw analyst ratings file
#     if os.path.exists(analyst_ratings_file):
#         print(f"Loading raw analyst ratings from: {analyst_ratings_file}")
#         raw_analyst_ratings = pd.read_csv(analyst_ratings_file)
    
#     return stock_data, raw_analyst_ratings

# # Call the function to load the data
# stock_data, raw_analyst_ratings = load_data()

# # Debug: Print available stock data keys
# print("Loaded stock data keys:", stock_data.keys())

# # Access and check raw analyst ratings
# if raw_analyst_ratings is not None:
#     print("\nRaw Analyst Ratings DataFrame Info:")
#     print(raw_analyst_ratings.info())

#     print("\nRaw Analyst Ratings Shape:")
#     print(raw_analyst_ratings.shape)

#     print("\nFirst few rows of Raw Analyst Ratings:")
#     print(raw_analyst_ratings.head())


# EDA

## Descriptive Statistics

In [30]:
# Check basic information about the DataFrame
print("DataFrame Info:")
raw_analyst_ratings.info()

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1407328 entries, 0 to 1407327
Data columns (total 6 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   Unnamed: 0  1407328 non-null  int64 
 1   headline    1407328 non-null  object
 2   url         1407328 non-null  object
 3   publisher   1407328 non-null  object
 4   date        1407328 non-null  object
 5   stock       1407328 non-null  object
dtypes: int64(1), object(5)
memory usage: 64.4+ MB


In [3]:
# # Check for null values
# print("\nNull Values per Column:")
# print(raw_analyst_ratings.isnull().sum())

Let's drop Unnamed: 0 column

In [4]:
# raw_analyst_ratings = raw_analyst_ratings.drop(columns=['Unnamed: 0'])


In [5]:
# print("DataFrame Info:")
# raw_analyst_ratings.info()

### Text Length Analysi:

In [6]:
# # Calculate basic statistics for headline length
# raw_analyst_ratings['headline_length'] = raw_analyst_ratings['headline'].apply(len)
# print("\nHeadline Length Statistics:")
# print(raw_analyst_ratings['headline_length'].describe())


### Publisher Frequency

In [7]:
# # Count the number of articles per publisher
# publisher_counts = raw_analyst_ratings['publisher'].value_counts()
# print("\nTop Publishers by Article Count:")
# print(publisher_counts.head(10))

### Publication Date Analysis

## Text Analysis (Sentiment Analysis)

we perform `sentiment analysis` on financial news headlines using **VADER (Valance Aware Dictionary anssEntiment Reasoner)** sentiment analysis tool.

In [36]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Apply sentiment analysis to the headlines
raw_analyst_ratings['sentiment_score'] = raw_analyst_ratings['headline'].apply(lambda x: analyzer.polarity_scores(x)['compound'])

# Categorize sentiment as Positive, Negative, or Neutral
raw_analyst_ratings['sentiment_category'] = raw_analyst_ratings['sentiment_score'].apply(
    lambda score: 'Positive' if score > 0.05 else ('Negative' if score < -0.05 else 'Neutral')
)

# Display sentiment statistics
print("\nSentiment Category Counts:")
print(raw_analyst_ratings['sentiment_category'].value_counts())



Sentiment Category Counts:
sentiment_category
Neutral     739338
Positive    442930
Negative    225060
Name: count, dtype: int64


## Time Series Analysis

In [37]:
print(raw_analyst_ratings['date'].head(20))


0     2020-06-05 10:30:54-04:00
1     2020-06-03 10:45:20-04:00
2     2020-05-26 04:30:07-04:00
3     2020-05-22 12:45:06-04:00
4     2020-05-22 11:38:59-04:00
5     2020-05-22 11:23:25-04:00
6     2020-05-22 09:36:20-04:00
7     2020-05-22 09:07:04-04:00
8     2020-05-22 08:37:59-04:00
9     2020-05-22 08:06:17-04:00
10          2020-05-22 00:00:00
11          2020-05-22 00:00:00
12          2020-05-21 00:00:00
13          2020-05-21 00:00:00
14          2020-05-21 00:00:00
15          2020-05-21 00:00:00
16          2020-05-18 00:00:00
17          2020-05-16 00:00:00
18          2020-05-15 00:00:00
19          2020-05-08 00:00:00
Name: date, dtype: object


In [38]:
# Convert 'date' column to datetime format
raw_analyst_ratings['date'] = pd.to_datetime(
    raw_analyst_ratings['date'], 
    format="%Y-%m-%d %H:%M:%S", 
    errors="coerce"
)


In [39]:
invalid_dates = raw_analyst_ratings[raw_analyst_ratings['date'].isna()]
print(f"Number of invalid dates: {len(invalid_dates)}")


Number of invalid dates: 55987


In [40]:
# Display a sample of the rows with invalid dates
print("Sample of invalid dates:")
print(invalid_dates[['date']].head(20))


Sample of invalid dates:
     date
0     NaT
1     NaT
2     NaT
3     NaT
4     NaT
5     NaT
6     NaT
7     NaT
8     NaT
9     NaT
1433  NaT
1434  NaT
1435  NaT
1436  NaT
1437  NaT
1438  NaT
1439  NaT
1440  NaT
1441  NaT
1442  NaT


In [41]:
total_rows = raw_analyst_ratings.shape[0]
invalid_dates_count = raw_analyst_ratings['date'].isna().sum()
invalid_date_percentage = (invalid_dates_count / total_rows) * 100

print(f"Total rows: {total_rows}")
print(f"Invalid dates: {invalid_dates_count} ({invalid_date_percentage:.2f}%)")


Total rows: 1407328
Invalid dates: 55987 (3.98%)


In [42]:
# Drop rows with NaT values in the 'date' column
raw_analyst_ratings = raw_analyst_ratings.dropna(subset=['date'])

# Verify the new shape of the dataset
print(f"Dataset after dropping invalid dates: {raw_analyst_ratings.shape}")


Dataset after dropping invalid dates: (1351341, 8)


In [43]:
invalid_dates = raw_analyst_ratings[raw_analyst_ratings['date'].isna()]
print(f"Number of invalid dates: {len(invalid_dates)}")


Number of invalid dates: 0


In [44]:
# Check for the earliest and latest publication dates
print("\nPublication Date Range:")
print(raw_analyst_ratings['date'].min(), "to", raw_analyst_ratings['date'].max())


Publication Date Range:
2009-02-14 00:00:00 to 2020-06-03 00:00:00


In [45]:
# Count the number of articles published per day
articles_per_day = raw_analyst_ratings.groupby(raw_analyst_ratings['date'].dt.date).size()
print("\nArticles Per Day:")
print(articles_per_day.head())


Articles Per Day:
date
2009-02-14    1
2009-04-27    2
2009-04-29    1
2009-05-22    1
2009-05-27    6
dtype: int64


In [46]:
# Identify the days with the highest number of publications
print("\nTop 5 Dates with the Most Articles:")
print(articles_per_day.nlargest(5))


Top 5 Dates with the Most Articles:
date
2020-03-12    1766
2020-02-27    1292
2020-02-28    1239
2019-08-01    1228
2020-03-06    1147
dtype: int64


In [47]:
# Identify unique publishers
unique_publishers = raw_analyst_ratings['publisher'].nunique()
print(f"\nNumber of Unique Publishers: {unique_publishers}")


Number of Unique Publishers: 1029


In [48]:
# Identify the most frequent domains (if publishers use email-style names)
if '@' in str(raw_analyst_ratings['publisher'].iloc[0]):  # Check for email-like names
    raw_analyst_ratings['publisher_domain'] = raw_analyst_ratings['publisher'].str.split('@').str[-1]
    domain_counts = raw_analyst_ratings['publisher_domain'].value_counts()
    print("\nTop 5 Publisher Domains:")
    print(domain_counts.head(5))

#Publisher Analysi

## Link Sentiment with Stock Prrice